##### Import

In [1]:
import warnings
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm
import shap
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
import os
import gc
import sys

# Filter out warning messages
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Set seaborn style
sns.set_style('whitegrid')

# Add the parent directory to sys.path
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Index and deciles for data slicing
idx = pd.IndexSlice

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
import pandas as pd
from pathlib import Path

DATA_STORE = Path('data/250_dataset.h5')

with pd.HDFStore(DATA_STORE) as store:
    keys = store.keys()

print(keys)

['/data/YEAR_20130102_20141208', '/data/YEAR_20141209_20161114', '/data/YEAR_20161115_20181022', '/data/YEAR_20181023_20200928', '/data/YEAR_20200929_20220902', '/data/YEAR_20220906_20230811', '/data/ic_based_reduced_features_YEAR_20220906_20230811']


In [ ]:
from utils import rank_and_quantize

with pd.HDFStore(DATA_STORE) as store:
    dataset = store['/data/YEAR_20220906_20230811']
    dataset = rank_and_quantize(dataset, TARGET_col='TARGET_ret_fwd_frac_order')

In [ ]:
dataset.info()

In [ ]:
dataset

In [ ]:
# Assuming your DataFrame is named df
cols = dataset.columns.tolist()

# Populate the features list with column names starting with 'feature_'
features = [col for col in cols if col.startswith('FEATURE_')]

# Find the first column starting with 'target_' and set it as the label
label_cols = [col for col in cols if col.startswith('TARGET_')]
# print(label_cols)
# label = label_cols[0] if label_cols else None
label = 'TARGET_ret_fwd_frac_order_quantiled'
print(len(features))  # This will show all the columns starting with 'feature_'
print(label)  # This will show the first column starting with 'target_'

In [ ]:
# Get unique dates and sort them
unique_dates = dataset.index.get_level_values('date').unique().sort_values()

# Adjust for the look-ahead gap
look_ahead = 1

# Split dates for training and testing with a gap
train_dates = unique_dates[:-21-look_ahead]
test_dates = unique_dates[-21:]

# Split the dataset
train_data = dataset.loc[pd.IndexSlice[:, train_dates], :]
test_data = dataset.loc[pd.IndexSlice[:, test_dates], :]

In [ ]:
def get_fi(model):
    fi = model.feature_importance(importance_type='gain')
    return (pd.Series(fi / fi.sum(),
                index=model.feature_name()))
                
def ic_lgbm(preds, train_data):
    """Custom IC eval metric for lightgbm"""
    is_higher_better = True
    return 'ic', spearmanr(preds, train_data.get_label())[0], \
        is_higher_better

In [ ]:
def sharpe_ratio_lgbm(preds, train_data):
    """Custom Sharpe ratio eval metric for lightgbm that calculates daily Spearman correlations."""
    labels = train_data.get_label()
    
    # Assuming the data index is a MultiIndex with date as the first level
    if not isinstance(train_data.data.index, pd.MultiIndex):
        raise ValueError("Expecting a MultiIndex with date as the first level")

    # Group by the first level of the MultiIndex (date) and compute the Spearman correlation for each group
    grouped_labels = pd.Series(labels, \
        index=train_data.data.index).groupby(level=0)
    # print(len(grouped_labels))
    grouped_preds = pd.Series(preds, \
        index=train_data.data.index).groupby(level=0)
    # print(len(grouped_preds))

    daily_scores = []
    for (_, actuals_for_day), (_, preds_for_day) in zip(grouped_labels, grouped_preds):
        score_for_day = spearmanr(actuals_for_day, preds_for_day)[0]
        if np.isnan(score_for_day):
            score_for_day = 0
        daily_scores.append(score_for_day)

    # Calculate the Sharpe ratio
    sharpe_ratio = np.mean(daily_scores) / (np.std(daily_scores) + 1e-9)  # added epsilon to avoid division by zero

    return 'sharpe_ratio', sharpe_ratio, True

In [ ]:
def metrics_on_fold(era_scores):
    era_scores = pd.Series(era_scores)
    
    # Calculate metrics
    mean_correlation = np.mean(era_scores)
    std_deviation = np.std(era_scores)
    sharpe_ratio = mean_correlation / std_deviation
    max_dd = (era_scores.cummax() - era_scores).max()

    # Smart Sharpe
    smart_sharpe = mean_correlation / (std_deviation + np.std(era_scores.diff()))
    
    # Autocorrelation
    autocorrelation = era_scores.autocorr()

    metrics = pd.Series({
        'mean_correlation': mean_correlation,
        'std_deviation': std_deviation,
        'sharpe_ratio': sharpe_ratio,
        'smart_sharpe': smart_sharpe,
        'autocorrelation': autocorrelation,
        'max_dd': max_dd,
        'min_correlation': era_scores.min(),
        'max_correlation': era_scores.max(),
    })

    # Cleanup
    _ = gc.collect()
    
    return metrics

In [ ]:
## DONT DELETE or MODIFY
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback
from scipy.stats import spearmanr
from utils import CustomBackwardMultipleTimeSeriesCV

warnings.filterwarnings('ignore')


evals_result = {}

def objective(trial, data, features, categoricals, cv):
    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'verbose': -1,
        'metric': 'None',
        'device': 'gpu',
        'num_leaves': trial.suggest_int('num_leaves', 30, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    }

    # Create a LightGBM callback for early stopping
    early_stopping = lgb.early_stopping(stopping_rounds=500, \
        verbose=True, first_metric_only=True)

    all_daily_scores = []
    for train_idx, val_idx in cv:
        train_features = data.loc[train_idx, features]
        train_labels = data.loc[train_idx, label]

        lgb_train = lgb.Dataset(data=train_features, label=train_labels, 
                                categorical_feature=categoricals,
                                free_raw_data=False)

        val_features = data.loc[val_idx, features]
        val_labels = data.loc[val_idx, label]

        lgb_val = lgb.Dataset(data=val_features, label=val_labels, 
                              categorical_feature=categoricals,
                              free_raw_data=False)

        model = lgb.train(params=params,
                          train_set=lgb_train,
                          num_boost_round=5000,
                          valid_sets=[lgb_train, lgb_val],
                          valid_names=['train', 'valid_0'],
                          feval=sharpe_ratio_lgbm,
                          callbacks=[lgb.record_evaluation(evals_result),
                                     early_stopping,
                                     LightGBMPruningCallback(trial, 'sharpe_ratio')])

        # Ensure your feval function returns daily scores for this to work correctly
        all_daily_scores.extend(evals_result['valid_0']['sharpe_ratio'])

    metrics = metrics_on_fold(all_daily_scores)

    # Now, return the smart_sharpe to be optimized by Optuna
    score = metrics['smart_sharpe'] #- 0.1 * metrics['max_dd']

    if np.isnan(score):
        return 1e-9  # Return a very low value if smart_sharpe is nan

    return score


uniques = dataset.nunique()

# categoricals = [col for col in uniques[uniques < 20].index if dataset[col].ge(0).all()]
categoricals = [col for col in uniques[uniques < 20].index if \
    dataset[col].ge(0).all() and col.startswith("feature_")]


print(f'Number of categorical vals: {len(categoricals)}')

cv = CustomBackwardMultipleTimeSeriesCV(train_data, train_period_length=142, 
                                        test_period_length=21, 
                                        lookahead=1, 
                                        date_idx='date')

def progress_bar(study, trial, n_trials):
    progress = (trial.number + 1) / n_trials
    best_trial_msg = ""
    if study.best_trial is not None:
        best_trial_msg = f"Best is trial {study.best_trial.number} with value: {study.best_trial.value}."
    print(f'Trial {trial.number + 1}/{n_trials} finished with value: {trial.value} and parameters: {trial.params}. {best_trial_msg}')

n_trials = 15
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=5))
study.optimize(lambda trial: objective(trial, train_data, features, categoricals, cv), 
               n_trials=n_trials, 
               callbacks=[lambda study, \
               trial: progress_bar(study, trial, n_trials)])

In [ ]:
cv_result = pd.DataFrame({'Train Set': evals_result['train']['sharpe_ratio'], 
                          'Validation Set': evals_result['valid_0']['sharpe_ratio']})

ax = cv_result.loc[:300].plot(figsize=(12, 4))
 
ax.axvline(cv_result['Validation Set'].idxmax(), c='k', ls='--', lw=1);

In [ ]:
best_params = study.best_params
print("Best parameters found by Optuna:")
print(best_params)

In [ ]:
lgb_train_all = lgb.Dataset(data=train_data[features], label=train_data[label], 
                            categorical_feature=categoricals, free_raw_data=False)

best_model = lgb.train(params=best_params,
                       train_set=lgb_train_all,
                       num_boost_round=5000,  # or some other number of boosting rounds
                       feval=sharpe_ratio_lgbm,
                       callbacks=[lgb.record_evaluation(evals_result)])

#### Test on unseen data

In [ ]:
test_features = test_data[features]
test_labels = test_data[label]

y_pred = best_model.predict(test_features)
# score = spearmanr(test_labels, y_pred)[0]
# print(f"Test set Spearman score: {score}")

preds = test_labels.reset_index(name='actual').assign(predicted=y_pred).set_index(['date', 'ticker'])
preds

In [ ]:
test_data

In [ ]:
test_features = test_data[features]
test_labels = test_data[label]

y_pred = best_model.predict(test_features)

preds = test_labels.reset_index(name='actual').assign(predicted=y_pred).set_index(['date', 'ticker'])

# Rename columns to add 'feature_' prefix
cols_to_rename = ['open', 'high', 'low', 'close', 'volume']
new_col_names = ["feature_" + col for col in cols_to_rename]
rename_dict = dict(zip(cols_to_rename, new_col_names))

test_data_renamed = test_data.rename(columns=rename_dict)

# Add ['feature_open', 'feature_high', 'feature_low', 'feature_close', 'feature_volume'] to preds
preds = preds.join(test_data_renamed[new_col_names], on=['date', 'ticker'])

preds

In [ ]:
def daily_spearman(group):
    return spearmanr(group['actual'], group['predicted'])[0]

daily_correlations = preds.groupby('date').apply(daily_spearman)

In [ ]:
# Calculate the mean and standard deviation of daily correlations
mean_daily_correlation = daily_correlations.mean()
std_daily_correlation = daily_correlations.std()

# Calculate Sharpe ratio for each date
daily_sharpe_ratios = (daily_correlations - mean_daily_correlation) / std_daily_correlation

In [ ]:
import matplotlib.pyplot as plt

# Create a list of colors based on the sign of the Sharpe Ratios
colors = ['blue' if value > 0 else 'red' for value in daily_sharpe_ratios]

plt.figure(figsize=(12,6))
daily_sharpe_ratios.plot(kind='bar', color=colors)
plt.title('Daily Sharpe Ratios')
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.grid(axis='y')
plt.tight_layout()
plt.axhline(y=0, color='black', linestyle='-')  # Here's where we add the horizontal line at y=0
plt.xticks(rotation=45)  # rotates the x-axis labels for better visibility
plt.show()

In [ ]:
# lr_r, lr_p = spearmanr(preds.actual, preds.predicted)
# print(f'Information Coefficient (overall): {lr_r:.3%} (p-value: {lr_p:.4%})')